In [1]:
import pandas as pd
import numpy as np
import re 
import string 
import warnings
import math

warnings.filterwarnings("ignore")


import torch
import os

from PIL import Image

import json
import matplotlib.pyplot as plt

import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
from time import sleep
import warnings
from transformers import AdamW

from transformers import CLIPProcessor, CLIPModel
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# read annotations file

Task1_Train = pd.read_csv("/kaggle/input/multimodal-daatset-for-memes-detection/Task1Multimodal Ready.csv")
Task1_Dev = pd.read_csv("/kaggle/input/multimodal-daatset-for-memes-detection/stA_eval(1).csv")
Task1_Test = pd.read_csv("/kaggle/input/multimodal-daatset-for-memes-detection/stA_test.csv")

Task1_Train.dropna(inplace=True)

In [3]:
Task1_Train['label'].value_counts()

label
1    1942
0    1658
Name: count, dtype: int64

In [4]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emoticons
                               u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
                               u"\U0001F680-\U0001F6FF"  # Transport & Map Symbols
                               u"\U0001F700-\U0001F77F"  # Alphanumeric Supplement
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_punctuation(input_string):
    translator = str.maketrans('', '', string.punctuation)
    
    result = input_string.translate(translator)
    
    return result

def remove_non_english_chars(input_string):
    translation_table = dict.fromkeys(
        i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('C')
    )

    result = input_string.translate(translation_table)

    return result

def correct_spelling(input_text):
    try:
        # Create a SpellChecker object
        spell = SpellChecker()

        # Split the input text into words
        words = input_text.split()

        # Find misspelled words
        misspelled = spell.unknown(words)

        # Correct misspelled words
        corrected_text = " ".join(spell.correction(word) if word in misspelled else word for word in words)

        return corrected_text

    except Exception as e:
        print(f"An error occurred: {e}")
        return input_text
    
hyperlink_pattern = r'https?://\S+|www\.\S+'

def PreProcessTweets(tweet):
  tweet2 = str(tweet)
  tweet2 = re.sub(r'^RT[\s]+', '', tweet)
  tweet2 = re.sub(r'#([^\s]+)', '', tweet2)
  tweet2 = tweet2.replace("LINK","")
  tweet2 = tweet2.replace("&amp","") 
  tweet2 = re.sub(r'@', '', tweet2)
  tweet2 = tweet2.replace("\n","")
  tweet2 = re.sub(r'_', ' ', tweet2)
  tweet2 = tweet2.replace("UN","United Nations ")
  tweet2 = re.sub(hyperlink_pattern, '', tweet2)
  tweet2 = tweet2.replace(":","") 
  tweet2 = re.sub('‼‼‼','', tweet2)
  tweet2 = remove_emojis(tweet2)
  tweet2 = remove_punctuation(tweet2)
  tweet2 = re.sub(' +', ' ',tweet2)
  tweet2 = tweet2.replace("youve","you have")
  tweet2 = tweet2.replace("govt","Government")
#   tweet2 = correct_spelling(tweet2)
  return tweet2

Task1_Train['text'] = Task1_Train['text'].apply(lambda x: PreProcessTweets(x))
Task1_Dev['text'] = Task1_Dev['text'].apply(lambda x: PreProcessTweets(x))
Task1_Test['text'] = Task1_Test['text'].apply(lambda x: PreProcessTweets(x))

In [5]:
ImagePathTest = '/kaggle/input/subtaska-hate-speach-detection/subtaskA-20231211T122329Z-001/subtaskA/'

for i in range(len(Task1_Test)):
    Task1_Test['filename'][i] = ImagePathTest + Task1_Test['filename'][i].split('/')[-1]

In [6]:
# Indvidual = '/kaggle/input/subtask-b/subTaskB-20231121T151628Z-001/subTaskB/Individual/'
# Community = '/kaggle/input/subtask-b/subTaskB-20231121T151628Z-001/subTaskB/Community/'
# Organization = '/kaggle/input/subtask-b/subTaskB-20231121T151628Z-001/subTaskB/Organization/'

HatePath = '/kaggle/input/task-a/subTaskA-20231121T171317Z-001/subTaskA/Hate Speech/'
NonHatePath = '/kaggle/input/task-a/subTaskA-20231121T171317Z-001/subTaskA/No Hate Speech/'

for i in range(len(Task1_Train)):
    if(Task1_Train['label'][i] == 0):
        Task1_Train['filename'][i] = NonHatePath + Task1_Train['filename'][i].split('/')[-1]
    elif(Task1_Train['label'][i] == 1):
        Task1_Train['filename'][i] = HatePath + Task1_Train['filename'][i].split('/')[-1]
#     elif(Task1_Train['label'][i] == 2):
#         Task1_Train['filename'][i] = Organization + Task1_Train['filename'][i].split('/')[-1]
 
DevPath = '/kaggle/input/subtaska-hate-speach-detection/subtaskA-20231111T125143Z-001/subtaskA/'

for i in range(len(Task1_Dev)):
    if(Task1_Dev['label'][i] == 0):
        Task1_Dev['filename'][i] = DevPath + Task1_Dev['filename'][i].split('/')[-1]
    else:
        Task1_Dev['filename'][i] = DevPath + Task1_Dev['filename'][i].split('/')[-1]

In [7]:
Task1_Train = pd.concat([Task1_Train,Task1_Dev],axis=0)
Task1_Train = Task1_Train.reset_index()

In [8]:
Task1_Train['label'].value_counts()

label
1    2185
0    1858
Name: count, dtype: int64

In [ ]:
from transformers import pipeline

pipe = pipeline("summarization", model="currentlyexhausted/flan-t5-summarizer", device = "cuda:0", max_length = 70) 

# for i in range(len(Task1_Train)):
#     if(len(Task1_Train['text'][i].split()) > 50):
#         Task1_Train['text'][i] = pipe(Task1_Train['text'][i])[0]['summary_text']

for i in range(len(Task1_Test)):
    if(len(Task1_Test['text'][i].split()) > 50):
        Task1_Test['text'][i] = pipe(Task1_Test['text'][i])[0]['summary_text']
                                                       
    #if(org_count > person_count):
        #Task1_Dev['label'][i] = 1

In [10]:
# DATASET

class Dataset(torch.utils.data.Dataset):
    """
    This is our custom dataset class which will load the text and their corresponding labels into Pytorch tensors
    """
    def __init__(self, paths, text, labels):
        self.labels = labels
        self.texts = text
        self.paths = paths

    def __getitem__(self, idx):
        sample = {}
        img_path = self.paths[idx]

        text = self.texts[idx]

        label = self.labels[idx]

        try:
            sample["label"] = int(label)
            sample["text"] = text
            sample["image"] = img_path
        except Exception as e:
            print(e)
        
        return sample
    
    def __len__(self):
        return len(self.labels)

In [11]:
Task1_Test['label'] = 0

dataset = Dataset(Task1_Train['filename'].values, Task1_Train['text'].values, Task1_Train['label'].values)

testdataset = Dataset(Task1_Test['filename'].values, Task1_Test['text'].values, Task1_Test['label'].values)

In [12]:
len(Task1_Train)

4043

In [17]:
# split into training, validation and testing

train_dataset, validation_dataset = torch.utils.data.random_split(dataset, [3643, 400])

In [18]:
import torch.nn.functional as F

class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()

        self.fc1 = nn.Linear(1792, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 3)  # Output layer for classification

        self.dropout = nn.Dropout(0.25)

    def forward(self, input):
        linear_out1 = self.dropout(F.relu(self.fc1(input)))
        linear_out2 = self.dropout(F.relu(self.fc2(linear_out1)))
        linear_out3 = self.dropout(F.relu(self.fc3(linear_out2)))
        final_out = self.fc4(linear_out3)  # Output layer with no activation for now

        return final_out

In [19]:
## Specify the Hyper parameters 

BATCH_SIZE = 8
NUM_LABELS = 2
EPOCHS = 10
LEARNING_RATE = 1e-5

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled   = True

In [20]:
## We call the dataloader class
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers=4,
    shuffle=True,
    drop_last=True
 )

test_loader = torch.utils.data.DataLoader(
    testdataset,
    batch_size=1,
    pin_memory=True,
    num_workers=0,
 )

val_loader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers=4,
    shuffle=True,
    drop_last=True
 )

dataloaders = {'Train': train_loader,'Test': test_loader, 'Val': val_loader}

In [21]:
Task1_Train['label'].value_counts()

label
1    2185
0    1858
Name: count, dtype: int64

In [22]:
clip = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")

processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

clip = clip.to(device)

model = Model().to(device)

#optimizer
optimizer = AdamW(model.parameters(), lr = LEARNING_RATE, eps=1e-8)

#Loss function
criterion = nn.CrossEntropyLoss()

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
import torch
from tqdm import tqdm
from PIL import Image

train_loss = []
train_acc = []

val_loss = []
val_acc = []

for epoch in range(0, EPOCHS):
    print('-' * 50)
    print('Epoch {}/{}'.format(epoch + 1, EPOCHS))

    for phase in ['Train', 'Val']:
        batch_loss = 0.0  # live loss
        batch_acc = 0.0  # live accuracy

        y_pred = []
        y_true = []

        if phase == 'Train':
            model.train()
        else:
            model.eval()

        with tqdm(dataloaders[phase], unit="batch", desc=phase) as tepoch:

            for idx, batch in enumerate(tepoch):
                labels = batch["label"].to(device)
                text = batch["text"]

                imgs = []

                img_paths = batch["image"]

                for path in img_paths:
                    imgs.append((Image.open(path)))

                inputs = processor(text=text, images=imgs, return_tensors="pt", padding=True, truncation=True)
                inputs = {key: value.to(device) for key, value in inputs.items()}
                clip_output = clip(**inputs)

                img_embed = clip_output.vision_model_output.pooler_output
                text_embed = clip_output.text_model_output.pooler_output

                concat = torch.cat((text_embed, img_embed), 1).to(device)

                output = model(concat)

                output = output.to(device)

                loss = criterion(output, labels)

                if phase == 'Train':
                    optimizer.zero_grad()

                    loss.backward()

                    optimizer.step()  

                batch_loss += loss.item()

                _, preds = output.data.max(1)

                y_pred.extend(preds.tolist())
                y_true.extend(labels.tolist())
                
                tepoch.set_postfix(loss=batch_loss / (idx + 1), accuracy=sum((y_pred[i] == y_true[i]) for i in range(len(y_true))) / len(y_true))

--------------------------------------------------
Epoch 1/10


Train:   0%|          | 0/455 [00:00<?, ?batch/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALL

--------------------------------------------------
Epoch 2/10


Train:   0%|          | 0/455 [00:00<?, ?batch/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALL

--------------------------------------------------
Epoch 3/10


...

Train:   0%|          | 0/455 [00:00<?, ?batch/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALL

--------------------------------------------------
Epoch 6/10


Train:   0%|          | 0/455 [00:00<?, ?batch/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALL

--------------------------------------------------
Epoch 7/10


Train:   0%|          | 0/455 [00:00<?, ?batch/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALL

In [45]:
y_pred = []
files = []

i = 0

with tqdm(dataloaders['Test'], unit="batch", desc=phase) as tepoch:
  for idx, batch in enumerate(tepoch):
    labels = batch["label"].to(device)
    text = batch["text"]

    imgs = []

    img_paths = batch["image"]

    for path in img_paths:
      i += 1
      files.append((path.split('/')[-1][:5]))
      imgs.append((Image.open(path)))
    
    inputs = processor(text=text, images=imgs, return_tensors="pt", padding=True, truncation = True)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    clip_output = clip(**inputs)

    img_embed = clip_output.vision_model_output.pooler_output

    text_embed = clip_output.text_model_output.pooler_output

    concat = torch.cat((text_embed, img_embed), 1)

    output = model(concat)

    _, preds = output.data.max(1)

    y_pred.extend(preds.tolist())

Val: 100%|██████████| 242/242 [00:18<00:00, 13.28batch/s]


In [47]:
pred = {}
for i in range(len(files)):
    pred[files[i]] = y_pred[i]
    
assert len(sorted_dict) == len(Task1_Test), "Check the inference loop."

In [49]:
import json

with open('submission.json', 'w') as json_file:
    for key, value in sorted_dict.items():
        predictionsDictionary = {'index': int(key), 'prediction': int(value)}
        json.dump(predictionsDictionary, json_file)    
        json_file.write('\n')    

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score

In [ ]:
accuracy_score(y_true,y_pred)

In [ ]:
f1_score(y_true, y_pred, average='macro')

In [ ]:
precision_score(y_true, y_pred, average='macro')

In [ ]:
recall_score(y_true, y_pred, average='macro')